# RHEED Simulation: MgO (001) Surface with <100> Beam

This tutorial demonstrates kinematic RHEED simulation for MgO following the approach in arXiv:2207.06642.

## Experimental Setup:
- **Crystal**: MgO (magnesium oxide, rock salt structure)
- **Surface**: (001) orientation
- **Beam direction**: <100> (along x-axis)
- **Electron energy**: 10 keV
- **Grazing angle**: 2.6°

## Expected Pattern:
For MgO (001) with <100> beam, we expect:
- Vertical streaks perpendicular to the beam
- Mirror symmetry about the vertical axis
- Streak spacing determined by surface reciprocal lattice

In [ ]:
import jax.numpy as jnp
import rheedium as rh

### Use `autoreload` to reload changed modules, you may disable this for normal use.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
structure_file = rh.inout.parse_cif("../tests/test_data/MgO.cif")

In [ ]:
structure_file

In [ ]:
# MgO (001) surface with <100> beam direction
# The CIF file already has (001) as the surface (z-direction)
# For RHEED: beam along <100> direction (x-axis) at grazing angle

zone_axis = jnp.array([0, 0, 1])  # (001) surface normal
beam_direction = jnp.array([1, 0, 0])  # <100> beam along x-axis

# Now, load the parameters

In [ ]:
# RHEED simulation parameters for MgO (001) surface, <100> beam
voltage_kV = 10.0  # Electron beam voltage in kV (typical for RHEED)
theta_deg = 2.6  # Grazing angle (1-3 degrees typical for RHEED)
phi_deg = 0.0  # Beam along x-axis (<100> direction)
hmax, kmax = 4, 4  # In-plane reciprocal lattice bounds
detector_distance = 80.0  # Detector distance in mm (typical RHEED geometry)
points_per_streak = 100  # Number of points to sample along each CTR streak

# Now, we will process things step by step.

## Electron wavelength (Å)

In [ ]:
lam_ang = rh.simul.wavelength_ang(voltage_kV)
lam_ang

## Build real-space lattice vectors

In [ ]:
cell_vectors = rh.ucell.build_cell_vectors(
    *structure_file.cell_lengths, *structure_file.cell_angles
)

In [ ]:
cell_vectors

## Generate reciprocal lattice vectors

In [ ]:
# For the streak simulator, we don't need to pre-generate reciprocal points
# The simulator handles CTRs internally. But we can still visualize the reciprocal lattice:
lmax = 2  # Just for visualization of bulk reciprocal lattice
Gs = rh.ucell.generate_reciprocal_points(
    crystal=structure_file, hmax=hmax, kmax=kmax, lmax=lmax, in_degrees=True
)
print(f"Reciprocal lattice points (for reference): {Gs.shape[0]}")

## Incident wavevector

In [ ]:
k_in = rh.simul.incident_wavevector(lam_ang, theta_deg)
k_in

## Build EwaldData for efficient angle scanning

The `build_ewald_data` function pre-computes all angle-independent quantities
(reciprocal lattice, structure factors with Kirkland form factors and Debye-Waller).
This enables efficient reuse when scanning beam angles.

In [ ]:
# Build EwaldData - pre-compute structure factors and reciprocal lattice
# This only needs to be done once per crystal/voltage combination
ewald_data = rh.simul.build_ewald_data(
    crystal=structure_file,
    voltage_kv=voltage_kV,
    hmax=hmax,
    kmax=kmax,
    lmax=lmax,
    temperature=300.0,  # Room temperature for Debye-Waller factors
)

print(f"Electron wavelength: {float(ewald_data.wavelength_ang):.4f} Å")
print(f"Wavevector magnitude |k|: {float(ewald_data.k_magnitude):.2f} 1/Å")
print(f"Number of reciprocal lattice points: {ewald_data.g_vectors.shape[0]}")
print(f"HKL grid shape: {ewald_data.hkl_grid.shape}")

## Find allowed reflections for specific beam angles

Now use `ewald_allowed_reflections` to find which reflections satisfy the Ewald 
sphere condition for a given beam orientation (theta, phi).

In [ ]:
# Find allowed reflections for theta=2°, phi=0° (beam along x-axis)
allowed_idx, k_out_allowed, intensities_allowed = rh.simul.ewald_allowed_reflections(
    ewald=ewald_data,
    theta_deg=theta_deg,
    phi_deg=phi_deg,
    tolerance=0.1,  # 10% tolerance for Ewald sphere intersection
)

# Filter out padding values (-1 indices)
valid_mask = allowed_idx >= 0
n_valid = int(jnp.sum(valid_mask))

print(f"Found {n_valid} allowed reflections at theta={theta_deg}°, phi={phi_deg}°")

# Show the allowed HKL indices
if n_valid > 0:
    valid_idx = allowed_idx[valid_mask]
    valid_hkl = ewald_data.hkl_grid[valid_idx]
    valid_I = intensities_allowed[valid_mask]
    print("\nAllowed reflections (h, k, l) and intensities:")
    for i in range(min(10, n_valid)):  # Show first 10
        h, k, l = valid_hkl[i]
        I = valid_I[i]
        print(f"  ({int(h):2d}, {int(k):2d}, {int(l):2d})  I = {float(I):.4f}")

## Azimuthal scan using pre-computed EwaldData

One major advantage of pre-computing EwaldData is efficient azimuthal scans.
The structure factors only need to be computed once!

In [ ]:
# Azimuthal scan: find reflections at different phi angles
phi_angles = jnp.linspace(0, 90, 10)  # Scan from 0° to 90°

print("Azimuthal scan results:")
print("-" * 40)
for phi in phi_angles:
    idx, k_out, intensities = rh.simul.ewald_allowed_reflections(
        ewald=ewald_data,
        theta_deg=theta_deg,
        phi_deg=float(phi),
        tolerance=0.1,
    )
    n_refl = int(jnp.sum(idx >= 0))
    total_I = float(jnp.sum(intensities[idx >= 0])) if n_refl > 0 else 0.0
    print(f"  phi = {float(phi):5.1f}°: {n_refl:3d} reflections, total I = {total_I:.2f}")

## Simulate RHEED spot pattern using discrete 3D reciprocal lattice

The `kinematic_spot_simulator` treats the reciprocal lattice as discrete 3D points.
This is useful for bulk-like diffraction or when only spot positions matter.
For grazing incidence RHEED, we need a larger `lmax` to capture reflections where
G_z exceeds the incident beam's z-component.

In [ ]:
# Generate RHEED spot pattern using discrete 3D reciprocal lattice
# kinematic_spot_simulator finds where discrete G points intersect the Ewald sphere
spot_pattern = rh.simul.kinematic_spot_simulator(
    crystal=structure_file,
    voltage_kv=voltage_kV,
    theta_deg=theta_deg,
    hmax=hmax,
    kmax=kmax,
    lmax=5,  # Need larger lmax for grazing incidence
    detector_distance=detector_distance,
    tolerance=0.05,
)

print(f"Number of spots: {len(spot_pattern.intensities)}")
print(f"X-coordinate range: [{spot_pattern.detector_points[:, 0].min():.2f}, {spot_pattern.detector_points[:, 0].max():.2f}] mm")
print(f"Y-coordinate range: [{spot_pattern.detector_points[:, 1].min():.2f}, {spot_pattern.detector_points[:, 1].max():.2f}] mm")

In [ ]:
# Plot the spot pattern
rh.plots.plot_rheed(spot_pattern, grid_size=300, interp_type="linear")

## Simulate RHEED streak pattern using Crystal Truncation Rods (CTRs)

The `kinematic_ctr_simulator` properly models RHEED as diffraction from a surface where
the reciprocal lattice consists of continuous rods rather than discrete points.
Each rod intersects the Ewald sphere along an arc, producing vertical streaks.

In [ ]:
# Generate RHEED streak pattern for MgO (001) with <100> beam
# kinematic_ctr_simulator models continuous crystal truncation rods (CTRs)
# and returns a RHEEDPattern directly
streak_pattern = rh.simul.kinematic_ctr_simulator(
    crystal=structure_file,
    voltage_kv=voltage_kV,
    theta_deg=theta_deg,
    hmax=hmax,
    kmax=kmax,
    detector_distance=detector_distance,
    n_points_per_rod=points_per_streak,
)

print(f"Number of streak points: {len(streak_pattern.intensities)}")
print(f"X-coordinate range: [{streak_pattern.detector_points[:, 0].min():.2f}, {streak_pattern.detector_points[:, 0].max():.2f}] mm")
print(f"Y-coordinate range: [{streak_pattern.detector_points[:, 1].min():.2f}, {streak_pattern.detector_points[:, 1].max():.2f}] mm")

## Check how many reflections were found

In [ ]:
# Summary of streak pattern
print(f"Number of streak points: {len(streak_pattern.intensities)}")
print(f"Number of unique rods: {len(jnp.unique(streak_pattern.G_indices))}")
print(f"Intensity range: [{streak_pattern.intensities.min():.3f}, {streak_pattern.intensities.max():.3f}]")
print("\nSample detector coordinates (first 5):")
print(streak_pattern.detector_points[:5])

In [ ]:
# Debug: Check which (h,k) rods passed the filter
print("\nRods included in pattern:")
unique_indices = jnp.unique(streak_pattern.G_indices)
for idx in unique_indices:
    h = int(idx // (2*kmax + 1)) - hmax
    k = int(idx % (2*kmax + 1)) - kmax
    count = int(jnp.sum(streak_pattern.G_indices == idx))
    print(f"  ({h:2d}, {k:2d}): {count} points")

## Visualize the RHEED pattern

The pattern should show vertical streaks characteristic of RHEED from (001) surface with <100> beam.

In [ ]:
rh.plots.plot_rheed(streak_pattern, grid_size=300, interp_type="linear")

In [ ]:
recip_vecs = rh.ucell.reciprocal_lattice_vectors(
    *structure_file.cell_lengths, *structure_file.cell_angles, in_degrees=True
)
atom_pos = structure_file.cart_positions[:, :3]
atom_Z = structure_file.cart_positions[:, 3].astype(jnp.int32)

test_cases = [
    (1, 0, 0),  # Should be ~0 (forbidden)
    (2, 0, 0),  # Should be ~6400 (allowed)
    (1, 1, 0),  # Should be ~0 (forbidden)
    (1, 1, 1),  # Should be non-zero (allowed)
    (0, 0, 0),  # Should be ~6400
]

for h, k, l in test_cases:
    G = h * recip_vecs[0] + k * recip_vecs[1] + l * recip_vecs[2]
    I = rh.simul.simple_structure_factor(G, atom_pos, atom_Z)
    print(f"I({h},{k},{l}) = {I:.1f}")

In [ ]:
lam = rh.simul.wavelength_ang(10.0)
k_in = rh.simul.incident_wavevector(lam, 2.0, 0.0)
recip_a = recip_vecs[0]
recip_b = recip_vecs[1]

print(f"k_in = {k_in}")
print(f"|k_in| = {jnp.linalg.norm(k_in):.2f}")

for h in [-2, 0, 2]:
    l_int, k_out, err = rh.simul.find_ctr_ewald_intersection(
        h=h, k=0, k_in=k_in, recip_a=recip_a, recip_b=recip_b
    )
    print(f"Rod ({h},0): l={float(l_int):.3f}, k_out_z={float(k_out[2]):.3f}, valid={jnp.isfinite(l_int) and k_out[2]>0}")